In [2]:
pip install pycoingecko

Note: you may need to restart the kernel to use updated packages.


In [3]:
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [9]:
import pandas as pd
import numpy as np
import time
from datetime import datetime

In [11]:
#import os
#os.getcwd()
#os.chdir("/Users/konstantin/Desktop/dateien/IRTG 1792/CoinGecko/coin_data")

'/Users/konstantin/Desktop/dateien/IRTG 1792/CoinGecko'

In [6]:
coinlist = cg.get_coins_list()
coinlist_df = pd.DataFrame(coinlist)
coinlist_df
# coinlist_df.to_csv("coinlist_test")
coin_ids = coinlist_df['id']

In [10]:
#this loop downloads all market data (=3 columns: price, market capitalization and volume) for each available coin, 
#   converts it into a data frame and saves it as a csv

#the problem of the output is that each element of the dataframe contains the timestamp and the value (instead of only
# the value)

for coin in coin_ids:
    tmp = cg.get_coin_market_chart_by_id(id=coin, vs_currency= 'usd', days='max') #returns a dictionary with 3 keys: price, market cap, volume
    tmp_df = pd.DataFrame(tmp)   #converts dictionary to dataframe
    tmp_df.to_csv(coin)  #saves dataframe as csv --> many csv-files (each one named with the coin name)
    time.sleep(20)

In [20]:
#the problem of the previous loop has been solved by this function: it creates a new dataframe which only contains the
# values and uses the timestamp (in a readable, rounded format) as row index
def create_dataframe(coin):

    from datetime import datetime 
    dataframe = []
    t = []
    names = [k for k, v in coin.items()]
    for i in range([len(v) for k, v in coin.items()][0]):
        y = [item[i][1] for item in coin.values()]
        x = [datetime.fromtimestamp(item[i][0]/1000).strftime("%Y-%m-%d") for item in coin.values()][0]
        dataframe.append(y)
        t.append(x)
    
    dataframe = pd.DataFrame(dataframe, columns = names)
    dataframe["Datetime"] = t
    dataframe["Datetime"] = pd.to_datetime(dataframe["Datetime"])
    dataframe = dataframe.set_index("Datetime")
    return dataframe


In [31]:
#the final loop 
for coin in coin_ids[0:3]:
    tmp = cg.get_coin_market_chart_by_id(id=coin, vs_currency= 'usd', days='max') #downloads market data for each coin
    result = create_dataframe(tmp) #the function from the cell above
    #result.to_csv(coin)
    print(result)
    time.sleep(20)

              prices   market_caps  total_volumes
Datetime                                         
2018-09-15  0.009788  30917.331417     103.397128
2018-09-16  0.009883  31330.589618      94.514808
2018-09-17  0.012145  38704.857566     307.498611
2018-09-18  0.007965  25529.340752     340.808326
2018-09-19  0.011866  38156.742601     166.514620
...              ...           ...            ...
2020-03-11  0.001318  14008.755344    3179.939230
2020-03-12  0.001320  14050.784111    3116.014642
2020-03-13  0.000775   8659.364900    1931.684114
2020-03-14  0.000927   9810.281631    2208.178918
2020-03-15  0.000941   9608.915926    2227.140596

[548 rows x 3 columns]
              prices  market_caps  total_volumes
Datetime                                        
2018-11-20  0.002838          0.0     253.534617
2018-11-21  0.003448          0.0    5384.450788
2018-11-22  0.002144          0.0     262.713559
2018-11-23  0.001139          0.0     267.817280
2018-11-24  0.001125          0.

In [ ]:
#now, merge datasets by Datetime